<a href="https://colab.research.google.com/github/PreamJ/LDA_Bud/blob/main/bud_lda02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

00 -> preprocess <br>
01 -> topic modeling (q/a/m) <br>
02 -> search

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
RANDOM_STATE = 42

In [ ]:
import pickle

with open('/content/drive/MyDrive/LDA_bud/preprocessed_data/data.pickle', 'rb') as f:
  data = pickle.load(f)
with open('/content/drive/MyDrive/LDA_bud/preprocessed_data/preprocessed_answer_data.pickle', 'rb') as f:
  preprocessed_answer_data = pickle.load(f)
with open('/content/drive/MyDrive/LDA_bud/preprocessed_data/preprocessed_question_data.pickle', 'rb') as f:
  preprocessed_question_data = pickle.load(f)
with open('/content/drive/MyDrive/LDA_bud/preprocessed_data/id2word.pkl', 'rb') as f:
  id2word = pickle.load(f)

In [ ]:
with open('/content/drive/MyDrive/LDA_bud/LDA_model/lda_model_30.pkl', 'rb') as f:
  lda_model_30 = pickle.load(f)

In [ ]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 46.3 MB/s eta 0:00:00


In [ ]:
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords

def preprocess(text):
    stopwords = list(thai_stopwords())
    read_stopwords = pd.read_excel('/content/drive/MyDrive/LDA_bud/stopwords/add_stopwords.xlsx')
    add_stopwords = read_stopwords['stopword'].values.tolist()
    result = []
    str_text = str(text).replace(' ','')
    word_token = word_tokenize(str_text, engine='newmm')
    for word in word_token:
        if(word not in stopwords + add_stopwords):
            result.append(word)
    return result
def word_tokenization(corpus):
  words_token = []
  for document in corpus:
    words_token.append(preprocess(document))
  return words_token


In [ ]:
import gensim
def bow(text):
  vector = id2word.doc2bow(text)
  return vector

In [ ]:
# from sklearn.model_selection import train_test_split
# train, test = train_test_split(preprocessed_question_data, test_size=0.2, random_state=RANDOM_STATE)

In [ ]:
# doc_topic = []
# for i in train:
#   doc_topic.append(lda_model_30.get_document_topics(bow(i)))
# len(doc_topic)

1733

In [ ]:
doc_topic = []
for i in preprocessed_question_data:
  doc_topic.append(lda_model_30.get_document_topics(bow(i)))
len(doc_topic)

2167

In [ ]:
from scipy.spatial import distance

In [ ]:
def jensenshannon(query):
  j_score = []
  # query = bow(preprocess(query))
  topic_prop = lda_model_30.get_document_topics(bow(query))
  for i in doc_topic:
    similarity = distance.jensenshannon(i, topic_prop)
    j_score.append(similarity[1])
  # print(j_score)
  j_score = list(enumerate(j_score))
  j_score = sorted(j_score, key=lambda x: x[1])
  max_10 = j_score[:10]

  similar_docs = []
  for i in range(10):
    similar_docs.append(data.iloc[j_score[i][0]])
  return similar_docs

In [ ]:
import random
x = random.randint(0,len(preprocessed_question_data)-1)
preprocessed_question_data[x]

['รัตน', 'ออกเสียง', 'ออกเสียง', 'ดีกว่า']

In [ ]:
preprocessed_question_data[x]

['รัตน', 'ออกเสียง', 'ออกเสียง', 'ดีกว่า']

In [ ]:
data.iloc[x]

question    สวดรัตนปรตรในใจไม่ออกเสียงได้ไหม หรือควรสวดออก...
answer      สวดไม่ออกเสียงก็ได้สวดในใจก็ได้ไม่เป็นไรไม่เป็...
Name: 750, dtype: object

In [ ]:
data.iloc[x]['question']

'สวดรัตนปรตรในใจไม่ออกเสียงได้ไหม หรือควรสวดออกเสียงดีกว่า'

In [ ]:
sim_docs = jensenshannon(preprocessed_question_data[x])
df = pd.DataFrame(sim_docs)
df[1:]

,question,answer
369,ตอนนี้หนู ท่องธาตุ อยู่แต่วันหนึ่ง หนูก็ฝันไปว...,ก็ท่องธาตุในระหว่างวันไป แล้วก็ก่อนนอนก็สวดบทอ...
1459,การท่องพระปริตร ให้ท่องเมื่อตอนที่โควิดแรงแรงท...,สวดบทรัตนปริตรนี้ในช่วงโควิดได้ทุกวันไม่ได้มีป...
183,การท่องรัตนปริตร และบทอื่นๆ ทําไมเราต้องท่อง 3...,พระอาจารย์ให้สวดย้ำ ๆ ซ้ำ ๆ 3-7 รอบ เพื่อต้องก...
1330,สวดรัตนปริตรและอาฏานาฏิยปริตร ท่องธาตุแล้วไม่ร...,ยินดีด้วย ทําบุญมาดี ดีมาก คนไม่รู้สึกอะไรนี่ดี
2085,ควรสวดมนต์อย่างไรถึงจะถูกต้องหรือสวดมนต์แล้วหา...,สวดมนต์แล้วหาว ใครก็ตามที่มีอาการอย่างนี้ขอให้...
907,คืออย่างที่เห็นทุกวันนี้ส่วนใหญ่นั้นจะมีพระพุท...,ถามว่าขาดไหม มันขึ้นอยู่กับว่าเขามีความเห็นเกี...
797,แล้วเวลาที่อาบน้ําแล้วเราเห็นที่มันออกมา ที่มั...,ก็เปลวมัน
1486,แล้วพระอาจารย์แม่ไม่ขาชาบ้างเหรอคะพระอาจารย์นั...,ไม่รู้สึกอะไรเลยไม่ขาไม่ชาไม่เจ็บไม่ปวดอาตมานั...
646,อยากทราบกำหนดเวลาไลฟ์คะ,พระอาจารย์ไม่ได้กำหนดเวลาที่แน่นอน หากมีการไลฟ...
